In [10]:
import pandas as pd
import numpy as np

read csv

In [11]:
df = pd.read_csv('..\\data\\sound_track_imdb_top_250_movie_tv_series.csv', delimiter=",", index_col=0)


In [12]:
df.columns

Index(['name', 'year', 'song_name', 'written_by', 'performed_by',
       'composed_by', 'lyrics_by', 'written_performed_by', 'music_by',
       'courtesy_of', 'conducted_by', 'libretto_by', 'under_license_from'],
      dtype='object')

select the needed columns and create new df

In [13]:
df_selected = df.loc[:,['name','year','song_name','performed_by']]
df_selected

,name,year,song_name,performed_by
0,The Shawshank Redemption,1994,If I Didn't Care,The Ink Spots
1,The Shawshank Redemption,1994,Duettino - Sull'aria,Edith Mathis (uncredited) and Gundula Janowitz...
2,The Shawshank Redemption,1994,Put The Blame On Mame,NaN
3,The Shawshank Redemption,1994,Lovesick Blues,Hank Williams
4,The Shawshank Redemption,1994,Willie and The Hand Jive,NaN
...,...,...,...,...
3128,"Love, Death & Robots",2019,It looks like we don't have any Soundtracks fo...,NaN
3129,Foyle's War,2002,It looks like we don't have any Soundtracks fo...,NaN
3130,Jesus of Nazareth,1977,It looks like we don't have any Soundtracks fo...,NaN
3131,Luther,2010,Paradise Circus,Massive Attack


In [14]:
# df_selected['performed_by'].isna().sum()

drop all movies/series that don't have any soundtrack

In [15]:
filter_mask = df_selected['song_name'] == "It looks like we don't have any Soundtracks for this title yet."
filter_mask.sum()


168

In [16]:
df_selected = df_selected[~filter_mask]
df_selected.reset_index(drop=True, inplace=True)
df_selected


,name,year,song_name,performed_by
0,The Shawshank Redemption,1994,If I Didn't Care,The Ink Spots
1,The Shawshank Redemption,1994,Duettino - Sull'aria,Edith Mathis (uncredited) and Gundula Janowitz...
2,The Shawshank Redemption,1994,Put The Blame On Mame,NaN
3,The Shawshank Redemption,1994,Lovesick Blues,Hank Williams
4,The Shawshank Redemption,1994,Willie and The Hand Jive,NaN
...,...,...,...,...
2960,Flames,2018,Khamoshiyan,Kumar Sanu and Alka Yagnik
2961,Modern Family,2009,Circle Of Life,Elton John
2962,Modern Family,2009,Modern Family Main Theme,NaN
2963,Modern Family,2009,We're All In This Together,Ty Burrell


Rename duplicate movie titles into unique ones

In [17]:
# Find the titles that exist in more than one year
movie_counts = df_selected.groupby('name')['year'].nunique()
movie_counts[movie_counts > 1].index
# movie_counts

Index(['Persona', 'The Office'], dtype='object', name='name')

In [18]:
duplicate_titles_df = df_selected[df_selected['name'].isin(movie_counts[movie_counts > 1].index)]
duplicate_titles_df

,name,year,song_name,performed_by
2497,Persona,1966,Adagio from Concerto No. 2 in E major for Viol...,NaN
2631,The Office,2005,The Office Theme,The Scrantones
2632,The Office,2005,Stars,NaN
2638,Persona,2018,Agâh Makami,Haluk Bilginer and Rubato
2874,The Office,2001,Handbags and Gladrags,Fin


In [19]:
df_selected[(df_selected.name == 'The Office') & (df_selected.year == 2005)]

,name,year,song_name,performed_by
2631,The Office,2005,The Office Theme,The Scrantones
2632,The Office,2005,Stars,NaN


In [20]:
df_selected.loc[(df_selected.name == 'The Office') & (df_selected.year == 2005), 'name'] = 'The Office (US)'

In [21]:
df_selected.loc[(df_selected.name == 'The Office') & (df_selected.year == 2001), 'name'] = 'The Office (UK)'

In [22]:
df_selected.loc[(df_selected.name == 'Persona') & (df_selected.year == 1966), 'name'] = 'Persona (1966)'

In [23]:
df_selected.loc[(df_selected.name == 'Persona') & (df_selected.year == 1966), 'name'] = 'Persona (2018)'

remove leading and trailing whitespace

In [24]:
df_selected = df_selected.map(lambda x: x.strip() if isinstance(x, str) else x)

In [25]:
# check if it was successful
mask_trailing_whitespace = df_selected.map(lambda x: isinstance(x, str) and x.strip() == x)
(~mask_trailing_whitespace).sum()

name               0
year            2965
song_name          0
performed_by    1304
dtype: int64

write df_selected to csv file

In [551]:
df_selected.to_csv("..\\data\\filtered_movie_songs_no_idx.csv", index=False)